Introduction
============

This is interactive notebook regarding "Introduction to path planning". (Author: Björn Hein)

Version | Authors
------------ | -------------
0.2 | Dennis McNab, Benjamin Dilly, Anton Kisel


License is based on Creative Commons: Attribution-NonCommercial 4.0 International (CC BY-NC 4.0) (pls. check: http://creativecommons.org/licenses/by-nc/4.0/)

This notebook imports all discussed algorithms and does a comparison

Important links are:

* General Info: http://www.python.org
* Python tutorial http://www.python.org/doc/tut/
* NetworkX http://networkx.github.io/
* NumPy and SciPy Guide http://docs.scipy.org/
* Matplotlib gallery http://matplotlib.sourceforge.net/gallery.html


Remember that:

* you have to press ctrl-return or shift-return, to execute the code in the code sections, only then the variables are "generated" and can be used
* you can execute the whole notebook by Cell->runAll

Adding all planners
===========


In [ ]:
from IPPerfMonitor import IPPerfMonitor
import IPBasicPRM
import IPVISBasicPRM

import IPVisibilityPRM
import IPVISVisibilityPRM

import IPLazyPRM
import IPVISLazyPRM

import SmoothBG
import SmoothGeneric

In [ ]:
import IPTestSuiteKin as ts

from shapely.geometry import Point, Polygon, LineString
from shapely import plotting

import matplotlib.pylab as plt
import matplotlib
%matplotlib inline

from IPEnvironmentKin import KinChainCollisionChecker, planarRobotVisualize, animateSolution

Set-up of the test scenario and the configuration for all planner
===================================

Following a procedure to compare all discussed planners are shown:

1. Configuration for every planner is defined
2. The configuration and the planner is stored in the variable setup, a Python dict()
3. the variable setup is then used to uniformly call the planning calls


In [ ]:
plannerFactory = dict()

basicConfig = dict()
basicConfig["radius"] = 3
basicConfig["numNodes"] = 200
#plannerFactory["basePRM"] = [IPBasicPRM.BasicPRM, basicConfig, IPVISBasicPRM.basicPRMVisualize]


basicConfig2 = dict()
basicConfig2["radius"] = 6
basicConfig2["numNodes"] = 600
# plannerFactory["basePRM2"] = [IPBasicPRM.BasicPRM, basicConfig2, IPVISBasicPRM.basicPRMVisualize]

basicConfig3 = dict()
basicConfig3["radius"] = 15
basicConfig3["numNodes"] = 300
#plannerFactory["basePRM3"] = [IPBasicPRM.BasicPRM, basicConfig3, IPVISBasicPRM.basicPRMVisualize]

visbilityConfig = dict()
visbilityConfig["ntry"] = 300
#plannerFactory["visibilityPRM"] = [IPVisibilityPRM.VisPRM, visbilityConfig, IPVISVisibilityPRM.visibilityPRMVisualize ]

lazyConfig = dict()
lazyConfig["initialRoadmapSize"] = 10
lazyConfig["updateRoadmapSize"]  = 5 
lazyConfig["kNearest"] = 8
lazyConfig["maxIterations"] = 20
plannerFactory["lazyPRM"] = [IPLazyPRM.LazyPRM, lazyConfig, IPVISLazyPRM.lazyPRMVisualize]

lazyConfig2 = dict()
lazyConfig2["initialRoadmapSize"] = 20
lazyConfig2["updateRoadmapSize"]  = 5 
lazyConfig2["kNearest"] = 15
lazyConfig2["maxIterations"] = 15
plannerFactory["lazyPRM2"] = [IPLazyPRM.LazyPRM, lazyConfig2, IPVISLazyPRM.lazyPRMVisualize]


In [ ]:
class ResultCollection (object):
    
    def __init__(self, plannerFactoryName, planner, benchmark, solution, perfDataFrame):
        self.bg_smoother = SmoothBG.SmoothBG()
        self.smoothing = SmoothGeneric.SmoothGeneric(planner, solution)
        configs = {}
        configs["corner_threshold"] = 0
        configs["collision_intervals"] = 200
        configs["max_deltree_depth"] = 10
        configs["min_deltree_delta"] = 0.05
        configs["epoches"] = 50
        self.plannerFactoryName = plannerFactoryName
        self.planner = planner
        self.benchmark = benchmark
        self.solution = solution
        self.perfDataFrame = perfDataFrame
        self.graph = planner.graph.copy()
        if solution != []:
            self.smoothed_path_generic, self.smooth_graph_generic = self.smoothing.smooth_path()
            self.smoothed_path_bg = self.bg_smoother.smooth_path(solution, planner, configs)
        else:
            self.smoothed_path_bg = []
            self.smoothed_path_generic = []
            self.smooth_graph_generic = []

In [ ]:
fullBenchList = ts.benchList

for benchmark in fullBenchList:
    print(benchmark.name)

In [ ]:
start_joint_pos = [2.0, 0.5]
end_joint_pos = [-2.0, -0.5]
for benchmark in fullBenchList:
    fig_local = plt.figure(figsize=(7,7))
    ax = fig_local.add_subplot(1,1,1)
    title = benchmark.name
    ax.set_title(title)
    ax.set_xlim([-4,4])
    ax.set_ylim([-4,4])
    try:
        benchmark.collisionChecker.drawObstacles(ax, True)
        benchmark.collisionChecker.kin_chain.move(start_joint_pos)
        (benchmark.collisionChecker.kin_chain, ax)
        benchmark.collisionChecker.kin_chain.move(end_joint_pos)
        planarRobotVisualize(benchmark.collisionChecker.kin_chain, ax)
    except Exception as e:
       print ("Error", e)
       pass

    

In [ ]:
resultList = list()
testList = fullBenchList

for key,producer in list(plannerFactory.items()):
    print(key, producer)
    for benchmark in testList:
        print ("Planning: " + key + " - " + benchmark.name)
        #planner = IPBasicPRM.BasicPRM(benchmark.collisionChecker)
        planner = producer[0](benchmark.collisionChecker)
        IPPerfMonitor.clearData()
        try:
            
            resultList.append(ResultCollection(key,
                                          planner, 
                                           benchmark, 
                                           planner.planPath(benchmark.startList,benchmark.goalList,producer[1]),
                                           IPPerfMonitor.dataFrame()
                                          ),
                        )
            print(resultList[-1].solution)
            print(resultList[-1].smoothed_path_generic)
            print(resultList[-1].smoothed_path_bg)
        except Exception as e:
        #    throw e
            print ("PLANNING ERROR ! PLANNING ERROR ! PLANNING ERROR ", e)
            pass
            


In [ ]:
matplotlib.rcParams['animation.embed_limit'] = 256
from IPEnvironmentKin import animateSolution
for result in resultList:
    if result.solution != []:
        # Animation Planned Path
        animateSolution(result.planner, result.graph, result.benchmark.collisionChecker, result.solution, IPVISLazyPRM.lazyPRMVisualize)
        # Animation Smoothing with BG
        animateSolution(result.planner, result.planner.graph, result.benchmark.collisionChecker, result.smoothed_path_bg, IPVISLazyPRM.lazyPRMVisualize, title="Smooth BG Path")
        # Animation Generic Smoothing
        animateSolution(result.planner, result.smooth_graph_generic, result.benchmark.collisionChecker, result.smoothed_path_generic, IPVISLazyPRM.lazyPRMVisualize, title="Smooth Generic Path")
    else:
        print(f"{result.plannerFactoryName} {result.benchmark.name} no Path found")

In [ ]:
import numpy as np
for bench in testList:
    title = bench.name
    pathLength = dict()
    planningTime = dict()
    roadmapSize = dict()
    smoothpathLength = dict()
    smoothpathLength_generic = dict()
    roadmapSizeBG = dict()
    roadmapSizeGeneric = dict()
    
    #try:
    for result in resultList:
        if result.benchmark.name == bench.name:
            #print result.benchmark.name  + " - " +  result.plannerFactoryName, len(result.solution)
            G = result.graph
            G_BG = result.planner.graph
            G_Generic = result.smooth_graph_generic
            path = result.solution
            coords = [(G.nodes[n]['pos'][0], G.nodes[n]['pos'][1]) for n in path]
            line = LineString(coords)
            pathLength[result.plannerFactoryName] = line.length
            smoothpath_bg = result.smoothed_path_bg
            smoothcoords = [(G_BG.nodes[n]['pos'][0], G_BG.nodes[n]['pos'][1]) for n in smoothpath_bg]
            smoothline = LineString(smoothcoords)
            smoothpath_generic = result.smoothed_path_generic
            smoothcoords_generic = [(G_Generic.nodes[n]['pos'][0], G_Generic.nodes[n]['pos'][1]) for n in smoothpath_generic]
            smoothline_generic = LineString(smoothcoords_generic)
            smoothpathLength_generic[result.plannerFactoryName] = smoothline_generic.length
            smoothpathLength[result.plannerFactoryName] = smoothline.length
            planningTime[result.plannerFactoryName] = result.perfDataFrame.groupby(["name"]).sum(numeric_only=True)["time"]["planPath"]
            roadmapSize[result.plannerFactoryName] = G.size()
            roadmapSizeBG[result.plannerFactoryName] = G_BG.size()
            roadmapSizeGeneric[result.plannerFactoryName] = len(G_Generic)


    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12, 8))

    labels = list(pathLength.keys())

    # -------- Daten --------
    planned_len   = np.array(list(pathLength.values()))
    bg_len        = np.array(list(smoothpathLength.values()))
    generic_len   = np.array(list(smoothpathLength_generic.values()))

    planned_rm    = np.array(list(roadmapSize.values()))
    bg_rm         = np.array(list(roadmapSizeBG.values()))
    generic_rm    = np.array(list(roadmapSizeGeneric.values()))

    n = len(labels)
    group_size = 6
    gap = 1.5
    width = 0.25

    # -------- X-Positionen mit Abstand nach je 6 --------
    x = []
    pos = 0
    for i in range(n):
        if i > 0 and i % group_size == 0:
            pos += gap
        x.append(pos)
        pos += 1
    x = np.array(x)

    # -------- Plot 1: Path Lengths --------
    ax1.bar(x - width, planned_len,  width, label="Planned path", color="blue")
    ax1.bar(x,         bg_len,       width, label="BG smoothing", color="green")
    ax1.bar(x + width, generic_len,  width, label="Generic smoothing", color="purple")

    ax1.set_ylabel("Path length")
    ax1.legend()
    ax1.grid(axis="y", linestyle="--", alpha=0.5)

    # -------- Plot 2: Roadmap Sizes --------
    ax2.bar(x - width, planned_rm,  width, label="Planned roadmap", color="blue")
    ax2.bar(x,         bg_rm,       width, label="BG roadmap", color="green")
    ax2.bar(x + width, generic_rm,  width, label="Generic roadmap", color="purple")

    ax2.set_ylabel("Roadmap size")
    ax2.legend()
    ax2.grid(axis="y", linestyle="--", alpha=0.5)

    # -------- X-Achse --------
    ax2.set_xticks(x)
    ax2.set_xticklabels(labels, rotation=45, ha="right")

    plt.tight_layout()
    plt.show()
